In [4]:
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage, Document  # Added Document here
from llama_index.embeddings.openai import OpenAIEmbedding
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# Load environment variables
load_dotenv()

# Get the API key from environment
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment. Please set it in your .env file.")

# Initialize LLM for LangChain
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

# Initialize embedding model for LlamaIndex
embed_model = OpenAIEmbedding(api_key=api_key)

# Directory paths
index_dir = "index_storage"
documents_dir = "docs"
persist_file = os.path.join(index_dir, "docstore.json")

# Ensure index directory exists
if not os.path.exists(index_dir):
    os.makedirs(index_dir)

# Check if index already exists
if not os.path.exists(persist_file):
    if not os.path.exists(documents_dir):
        os.makedirs(documents_dir)
        raise FileNotFoundError(f"No documents found in {documents_dir}. Please add documents to index.")
    
    documents = SimpleDirectoryReader(documents_dir).load_data()
    storage_context = StorageContext.from_defaults()
    vector_index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        embed_model=embed_model
    )
    vector_index.storage_context.persist(persist_dir=index_dir)
    print(f"Created and persisted new index at {index_dir}")
else:
    storage_context = StorageContext.from_defaults(persist_dir=index_dir)
    vector_index = load_index_from_storage(
        storage_context,
        embed_model=embed_model
    )
    print(f"Loaded existing index from {index_dir}")

Created and persisted new index at index_storage


In [5]:
# Create a query engine for retrieving context from the index
query_engine = vector_index.as_query_engine()

In [ ]:
# Default Scrum-specific prompts (used as fallback if index query fails)
default_prompts = {
    "Customer Proxy": """You are the Customer Proxy. Relay the following customer input to the Product Owner exactly as provided: {input}""",
    "Product Owner": """You are the Product Owner. Convert the following requirements into a prioritized backlog of user stories: {requirements}
1. Break down the requirements into clear user stories (e.g., "As a [role], I want [feature] so that [benefit]").
2. Assign story points (1, 2, 3, 5, 8) based on effort and complexity.
3. Prioritize based on business value.
4. Format as a numbered list.""",
    "Scrum Master": """You are the Scrum Master. Create a sprint plan from the following backlog: {backlog}
1. Select user stories totaling up to 20 story points for a 2-week sprint.
2. Assign tasks to team roles (e.g., Developers, QA).
3. Provide a brief sprint goal.
4. Format as a structured plan.""",
    "Development Team": """You are the Development Team. Execute the following sprint plan: {sprint_plan}
1. For each user story, estimate effort (e.g., SLOC for Developers, test cases for QA).
2. Produce deliverables (e.g., code snippets, test cases).
3. Format as a list of deliverables with effort estimates."""
}

In [ ]:
# Define dynamic prompt templates with fallback
def get_agent_prompt(agent_name, input_var, input_data):
    # Query the index for the agent's role and instructions
    role_query = f"What is the role and system message for the {agent_name} agent?"
    role_info = query_engine.query(role_query).response
    
    # Check if the query returned useful Scrum-specific info; if not, use default
    if "Scrum" not in role_info and "sprint" not in role_info.lower():
        prompt = default_prompts[agent_name]
    else:
        prompt = f"{role_info}\n\nBased on this role, process the following input:\n{{{input_var}}}: {input_data}"
    
    return PromptTemplate(input_variables=[input_var], template=prompt)

In [7]:
# Define the sprint function with index-driven prompts
def run_sprint(customer_input, sprint_number):
    print(f"\n--- Sprint {sprint_number} ---")
    
    # Customer Proxy
    customer_prompt = get_agent_prompt("Customer Proxy", "input", customer_input)
    customer_chain = LLMChain(llm=llm, prompt=customer_prompt, memory=ConversationBufferMemory())
    requirements = customer_chain({"input": customer_input})["text"]
    vector_index.insert(Document(text=requirements, metadata={"type": "requirements", "sprint": sprint_number}))
    print("Customer Proxy Output:", requirements)
    
    # Product Owner
    product_owner_prompt = get_agent_prompt("Product Owner", "requirements", requirements)
    product_owner_chain = LLMChain(llm=llm, prompt=product_owner_prompt, memory=ConversationBufferMemory())
    backlog = product_owner_chain({"requirements": requirements})["text"]
    vector_index.insert(Document(text=backlog, metadata={"type": "backlog", "sprint": sprint_number}))
    print("Product Owner Backlog:", backlog)
    
    # Scrum Master
    scrum_master_prompt = get_agent_prompt("Scrum Master", "backlog", backlog)
    scrum_master_chain = LLMChain(llm=llm, prompt=scrum_master_prompt, memory=ConversationBufferMemory())
    sprint_plan = scrum_master_chain({"backlog": backlog})["text"]
    vector_index.insert(Document(text=sprint_plan, metadata={"type": "sprint_plan", "sprint": sprint_number}))
    print("Scrum Master Sprint Plan:", sprint_plan)
    
    # Development Team
    dev_team_prompt = get_agent_prompt("Development Team", "sprint_plan", sprint_plan)
    dev_team_chain = LLMChain(llm=llm, prompt=dev_team_prompt, memory=ConversationBufferMemory())
    deliverables = dev_team_chain({"sprint_plan": sprint_plan})["text"]
    vector_index.insert(Document(text=deliverables, metadata={"type": "deliverables", "sprint": sprint_number}))
    print("Development Team Deliverables:", deliverables)
    
    # Customer Feedback
    feedback = input(f"Review deliverables for Sprint {sprint_number}:\n{deliverables}\nEnter feedback: ")
    vector_index.insert(Document(text=feedback, metadata={"type": "feedback", "sprint": sprint_number}))
    print("Customer Feedback:", feedback)
    return feedback

In [8]:
# Run 3 sprints
initial_requirements = "We need a system with real-time inventory tracking, automated reordering, and demand forecasting."
feedback = initial_requirements
for sprint in range(1, 4):
    feedback = run_sprint(feedback, sprint)
vector_index.storage_context.persist(persist_dir=index_dir)


--- Sprint 1 ---


/tmp/ipykernel_1668/2292546111.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  customer_chain = LLMChain(llm=llm, prompt=customer_prompt, memory=ConversationBufferMemory())
/tmp/ipykernel_1668/2292546111.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  customer_chain = LLMChain(llm=llm, prompt=customer_prompt, memory=ConversationBufferMemory())
/tmp/ipykernel_1668/2292546111.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  requirements = customer_chain({"input": customer_input})["text"]


Customer Proxy Output: To facilitate the request regarding the system requirements, I will convey the following information to the relevant system team:

---

**Customer Request Summary:**

The customer requires a system with the following features:

1. **Real-Time Inventory Tracking**: The system should be capable of monitoring inventory levels continuously and updating stock status instantly.
  
2. **Automated Reordering**: The system must include functionality to automatically reorder stock once it reaches predefined thresholds to prevent shortages.
  
3. **Demand Forecasting**: The system should provide the ability to analyze historical data and predict future demand to optimize inventory levels.

---

I will ensure that these requirements are properly conveyed and handled by the appropriate teams for further assessment and implementation.
Product Owner Backlog: As the Product Owner, it's crucial to prioritize the features based on their business value while ensuring that the devel

In [9]:
# Test query to verify the index
response = query_engine.query("What was the feedback for Sprint 1?")
print("Query Response:", response)

Query Response: The feedback for Sprint 1 was likely discussed on page 11 of the document located at "/home/ruegen/SPM-Retail-Project/phase_2/docs/retail pdf.pdf".
